In [ ]:
import pandas as pd
import sqlite3
from newsapi import NewsApiClient
import logging
news_api=NewsApiClient(api_key="7bcfaa6d966c420499afe97225ed186b")
from datetime import datetime, timedelta, time
from airflow import DAG
from airflow.operators.python import PythonOperator

In [ ]:
def extract():
    try:
        result = news_api.get_everything(q = "AI", language="en",sort_by = "publishedAt")
        logging.info("Connection is successful.")
        return result["articles"]
    except:
        logging.error("Connection Unsuccessful")
        return 
    
articles = extract()
print(articles)

In [ ]:
def cleanauthor_column(text):
    try:
        text.split(',')[0].title()
    except:
        print("No author")

In [ ]:
def transform_news_data(articles):
        article_list=[]
        for i in articles:
                article_list.append([val.get("name",0) if  key =='source' else val for key,val in i.items() if key in ["author","title","publishedAt","content","url","source"]])
                df = pd.DataFrame(article_list,columns = ["Source",'Author Name','News Title','URL','Date Published','Content'])
                df['Date Published'] = pd.to_datetime(df['Date Published']).dt.strftime('%Y-%m-%d %H:%M:%S')
                df["Author Name"] = df["Author Name"].apply(cleanauthor_column)
                return df
transformed = transform_news_data(articles)
print(transformed)

In [ ]:
def load_news_data(data):
    with sqlite3.connect("/usercode/news_data.sqlite") as connection:
        cursor = connection.cursor()
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS news(
                "Source" VARCHAR(30),
                "Author Name" Varchar(50),
                "News Title" text,
                "URL" text,
                "Date Published" Text,
                "Content" Text
            )
            
            ''')
        data.to_sql(name = "news",con = connection, index= False, if_exists = "append")
load_news_data(transformed)


In [ ]:
with sqlite3.connect("/usercode/news_data.sqlite") as connection:
    df = pd.read_sql("SELECT * FROM news;", connection)
df.head()